# <ins>Analysis on the Potential of Life on Exoplanets</ins>

## By: Ethan Barr and Tim Freerksen

## <ins>Introduction</ins>

Is there life in space? This has been a question for many years with no real evidence to back up the claim that there is. Our goal in this project is to take the exoplanets already discovered and see what the probability of life on these planets real is and hope that a good amount of the exoplanets that we discovered so far fit well into the criteria of haveing the ability to hold life.

Throughout this tutorial, we will try to see how many of the exoplanets we have discovered so far the right conditions in order to harbor life and then we will see what the probability is that a future exoplanet will end up holding life.

### <ins>Required Tools</ins>

The following libraries used for the project:

    1. pandas
    2. 

If having issues with python 3+ or panda, we recommend referring to these following websites for more information:

    1. https://docs.python.org/3/
    2. https://pandas.pydata.org/pandas-docs/stable/install.html

### <ins>1. Data Collection</ins>

This is the first part of the data life cycle. In this part we will go through various websites to try and find data that both matches our topic at hand as well as gives enough information so that we can perform an analysis later on. 

For an Exoplanet Database we found that https://exoplanetarchive.ipac.caltech.edu/cgi-bin/TblView/nph-tblView?app=ExoTbls&config=PS gave the best and most information. In order to retrieve this data we first converted the online database into a csv file in which we could then read and manipulate.

The following tools were used for the data collection:

    1. panda

In [1]:
import pandas as pd # used in order to read the csv file and convert it successfully into a datafram

Since the exoplanetarchive website was nice enough to allow the downloading of the database into a csv file, there was not a lot of steps to fully access the entire database. It was sufficient to first download the database in a csv format and then add it as one of the files with this project. We could then easily access this file by performing a pandas read_csv which allowed for the entire csv file to be converted into a flexibile and readable DataFrame that we could use.

A <ins>DataFrame</ins> is a table that has rows and columns that correlate to certain pieces of data. Using DataFrames allows for better use of more pandas functions which help to manipulate this data much more flexibily. If interested in learning more about DataFrames then check out the pandas documentation of it at https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html

The only issue that the csv had when we first read it was that the data provided had its only numbering associated with each row. To combact this we decided to stick with how the original data numbered itself and assigned that row to be the row number titled, loc_rowid.



In [2]:
data = pd.read_csv('PS.csv')
data.set_index('loc_rowid', inplace=True)
data.head()

,pl_name,hostname,default_flag,sy_snum,sy_pnum,discoverymethod,disc_year,disc_facility,soltype,pl_controv_flag,...,sy_vmagerr2,sy_kmag,sy_kmagerr1,sy_kmagerr2,sy_gaiamag,sy_gaiamagerr1,sy_gaiamagerr2,rowupdate,pl_pubdate,releasedate
loc_rowid,,,,,,,,,,,,,,,,,,,,,
1,11 Com b,11 Com,0,2,1,Radial Velocity,2007,Xinglong Station,Published Confirmed,0,...,-0.023,2.282,0.346,-0.346,4.44038,0.003848,-0.003848,2014-07-23,2011-08,2014-07-23
2,11 Com b,11 Com,1,2,1,Radial Velocity,2007,Xinglong Station,Published Confirmed,0,...,-0.023,2.282,0.346,-0.346,4.44038,0.003848,-0.003848,2014-05-14,2008-01,2014-05-14
3,11 UMi b,11 UMi,0,1,1,Radial Velocity,2009,Thueringer Landessternwarte Tautenburg,Published Confirmed,0,...,-0.005,1.939,0.270,-0.270,4.56216,0.003903,-0.003903,2018-04-25 14:08:01,2009-10,2014-05-14
4,11 UMi b,11 UMi,0,1,1,Radial Velocity,2009,Thueringer Landessternwarte Tautenburg,Published Confirmed,0,...,-0.005,1.939,0.270,-0.270,4.56216,0.003903,-0.003903,2018-04-25 14:08:01,2011-08,2014-07-23
5,11 UMi b,11 UMi,1,1,1,Radial Velocity,2009,Thueringer Landessternwarte Tautenburg,Published Confirmed,0,...,-0.005,1.939,0.270,-0.270,4.56216,0.003903,-0.003903,2018-09-04 16:14:36,2017-03,2018-09-06


### <ins>2. Data Processing</ins>

After you successfully retrieve the data that you are looking for and have it in some sort of dataframe so that you can manipulate it then you move onto this next step. Within this step we want to try and tidy up the data that we just read in. This is an important step because of the fact that it will allow the data to be read and understood with much more fluidity. In our case we would be altering the structure of the DataFrame through the process of tidying data and / or data wrangling.

You can learn more about:

    1. tidying data: https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html
    2. data wrangling: https://www.elderresearch.com/blog/what-is-data-wrangling-and-why-does-it-take-so-long/#:~:text=Data%20wrangling%20is%20the%20process,20%25%20for%20exploration%20and%20modeling.

We will now go through the steps of tiding up our DataFrame so that any extra information is scrap the columns that are repeated but with slightly different values due to the units that they are being shown in. We also want to convert the units that a few columns are using in order to allow for calculations to be performed a lot quickly and with ease.